In [1]:
!pip install --upgrade --quiet pinecone pinecone-text pinecone-notebooks
!pip install langchain-community
!pip install langchain_groq
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible

In [34]:
from langchain_groq import ChatGroq
import PyPDF2 as pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec
from pinecone_text.sparse import BM25Encoder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

In [3]:
llm = ChatGroq(
    model = "llama-3.3-70b-versatile",
    groq_api_key = "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",
    temperature = 0
)

In [5]:
def get_text(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = pdf.PdfReader(pdf_path)
        texts = ""
        for page in reader.pages:
            extracted = page.extract_text()
            if extracted:
                texts += extracted
        return texts

In [7]:
def get_chunks(texts):
    splitter = RecursiveCharacterTextSplitter(
        separators = ["\n\n", "\n", " "],
        chunk_size = 1500,
        chunk_overlap = 150
    )
    chunks = splitter.split_text(texts)
    return chunks

In [8]:
texts = get_text("/kaggle/input/ahkame-rooze/rooze.pdf")

In [9]:
chunks = get_chunks(texts)

In [13]:
embeddings = HuggingFaceEmbeddings(model_name = "myrkur/sentence-transformer-parsbert-fa-2.0")

/tmp/ipykernel_36/3678157302.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "myrkur/sentence-transformer-parsbert-fa-2.0")
2025-09-03 18:13:13.318695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756923193.688968      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756923193.787800      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempti

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [18]:
api_key = "pcsk_4xg4bN_LuDMufaBZ8tBCnWTxgg2fiuEAV7Y5kJksqnHHVZSFsoqYEW4RSPrJnNTBktymxh"
index_name = "hybrid-search-langchain-pinecone2"
pc = Pinecone(api_key = api_key)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 768,
        metric = "dotproduct",
        spec = ServerlessSpec(cloud = "aws", region = "us-east-1")
    )

In [19]:
bm25_encoder = BM25Encoder().default()

In [20]:
index = pc.Index(index_name)
index

In [22]:
bm25_encoder.fit(chunks)
bm25_encoder.dump("bm25_values.json")
bm25_encoder = BM25Encoder().load("bm25_values.json")

  0%|          | 0/28 [00:00<?, ?it/s]

In [23]:
retriever = PineconeHybridSearchRetriever(embeddings = embeddings, sparse_encoder = bm25_encoder, index = index)

In [29]:
retriever.add_texts(chunks)

  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
retriever.invoke("روزه در چه صورت باطل می شود؟")

[Document(metadata={'score': 211.661301}, page_content='خودش افطار کند، مثالً به او بگویند اگر غذا نخوری ضرر جانی یا مالی به تو می زنیم و او خودش برای  \nجلوگیری از ضرر، غذا بخورد) روزه اش باطل می شود. \n اگر روزه دار سهواً یکی از کارهایی را بکند که موجب باطل شدن روزه است و سپس به خیال این که  \nروزه اش باطل شده، عمداً بار دیگر یکی از آن کارها را انجام دهد، روزه ی او باطل می شود. \n هرگاه کسی شک کند مفطری انجام داده است  یا نه، مثالً غبار غلیظی که وارد دهان شده آن را فروبرده  \nاست یا نه، یا آبی که داخل دهان کرده بیرون ریخته یا نه، روزه اش صحیح است. ۸ .کفاره  ی افطار عمدی  \n روزهی ماه مبارک رمضان   \n۱ .وجوب کفاره و موارد آ  ن \nالف) هرگاه در ماه مبارک رمضان کارهایی که روزه را باطل می کند را از روی عمد و اختیار و بدون عذ ر  \nشرعی انجام دهد، عالوه بر این که روزه ی او باطل می شود و قضا دارد کفاره نیز بر او واجب است اعم از این  \nکه هنگام انجام مفطر، عالم به وجوب کفاره باشد یا نه   . \nتوجه : \n اگر فردی به خاطر عذری احتمال دهد که روزه ی ماه مبارک رمضان بر او واجب نیست و به همین دلیل  \

In [35]:
prompt = ChatPromptTemplate.from_template(
    """
    فقط بر اساس متن ارائه شده، به سوال پاسخ بده.
    متن: {context}
    سوال: {input}
    """
)

In [36]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [37]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [39]:
print(retrieval_chain.invoke({"input": "روزه به چه مناست؟"}))

{'input': 'روزه به چه مناست؟', 'context': [Document(metadata={'score': 159.055527}, page_content='۱ .معنای روز ه  \n«    در شرع مقدس اسالم آن است که انسان در تمام روز (از طلوع فجر تا غروب آفتاب) با قصد اطاعت»روزه\nاز فرمان خداوند از خوردن و آشامیدن و چیزهای دیگری که به تفصیل گفته خواهد شد خودداری کند . \nتوجه: \n معیار شرعی در مورد وقت روزه، فجر صادق است نه کاذب، و احراز آن موکول به تشخیص مکلف است. \n در طلوع فجر (وقت وجوب امساک برای روزه گرفتن) فرقی بین شبهای مهتابی و غیر مهتابی نیست . \n  جهت رعایت احتیاط در امساک روزه، همزمان با  »اَیَّدَهُمُ اهللُ تَعَالی« مقتضی است که مؤمنین محترم\nشروع اذان صبح از رسانه ها برای روزه امساک نمایند . \n هرگاه برای روزه دار اطمینان حاصل شود که اذان از هنگام داخل شدن وقت شروع شده جایز است به  \nصرف شروع اذان، افطار نماید و صبر کردن تا پایان اذان الزم نیست. \n \n ۲ .انواع روز ه  \nروزه از یک نظر بر چهار نوع است: \n۱ .روزه ی واجب، مثل روز ه ی ماه مبارک رمضان. \n۲ .روزه ی مستحب، مثل روزهی ماه رجب و شعبان . \n۳ .روز ه ی مکروه، مثل روزه ی روز عاشورا. \n۴ .

In [41]:
def generate_response(query):
    response = retrieval_chain.invoke({"input": query})
    return response["answer"]

In [43]:
print(generate_response("در چه صورت روزه باطل می شود؟"))

روزه در صورت انجام کارهایی که روزه را باطل می‌کند، مانند خوردن، آشامیدن، جماع و غیره، باطل می‌شود. همچنین اگر روزه‌دار عمداً و از روی اختیار کاری را انجام دهد که روزه را باطل می‌کند، روزه او باطل می‌شود.


In [44]:
print(generate_response("روزه بر چه افرادی واجب نیست؟"))

روزه بر افرادی واجب نیست که واجد شرایط وجوب روزه نباشند. این شرایط عبارتند از:
بلوغ
عقل
قدرت
بی هوش نبودن
مسافر نبودن
حایض و نفسا نبودن
ضرری نبودن روزه
حرجی نبودن روزه

بنابراین روزه بر کودک نابالغ، دیوانه، بیهوش، فردی که توانایی گرفتن روزه ندارد، مسافر، زن حایض و نفسا، کسی که روزه برای او ضرر یا حرج (مشقت زیاد) دارد واجب نیست.


In [46]:
print(generate_response("مفطرات روزه را توضیح بده"))

مفطرات روزه به کارهایی گفته می‌شود که روزه را باطل می‌کند. در متن ارائه شده، ۹ مورد از مفطرات روزه ذکر شده است:

1. خوردن و آشامیدن: اگر روزه‌دار عمداً و آگاهانه چیزی بخورد یا بیاشامد، روزه‌اش باطل می‌شود.
2. جماع (آمیزش جنسی)
3. استمنا (خود ارضایی جنسی)
4. دروغ بستن به خدا و پیامبران و معصومین (بنابر احتیاط واجب)
5. رساندن غبار غلیظ به حلق (بنابر احتیاط واجب)
6. فرو بردن تمام سر در آب (بنابر احتیاط واجب)
7. باقی ماندن بر جنابت و حیض و نفاس تا اذان صبح
8. اماله (تنقیه) کردن با مایعات
9. قی کردن عمدی
